In [ ]:
%pip install matplotlib pajndas scipy

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Parameters met aannemelijke marges
parameters = ["Rendement Zonnepaneel (%)", "Temperatuurscoëfficiënt (%)"]

# Simuleer tijd en temperatuurdata
np.random.seed(42)
time_steps = np.linspace(0, 24, 100)  # 100 tijdstappen over 24 uur
temperature = 25 + 10 * np.sin(time_steps / 24 * 2 * np.pi)  # Dagelijkse temperatuurvariatie (°C)

# Realistische basis voor gemeten vermogen (kW)
true_efficiency = 0.21  # 21% paneelrendement
true_temp_coeff = -0.004  # -0.4% verlies per graad boven STC
measured_power = (
    3.0 * np.exp(-0.1 * (time_steps - 12) ** 2)  # Realistisch dagprofiel (in kW)
) * true_efficiency * (1 + true_temp_coeff * (temperature - 25))
measured_power *= np.random.uniform(0.95, 1.05, len(time_steps))  # Ruis toevoegen

# Simuleer een "foutieve" simulatie met veel lagere overeenkomst
baseline_efficiency = 0.19  # 19% rendement
baseline_temp_coeff = -0.003  # -0.3% verlies per graad
simulated_power = (
    2.8 * np.exp(-0.15 * (time_steps - 12) ** 2)
) * baseline_efficiency * (1 + baseline_temp_coeff * (temperature - 25))

# Simulatiefunctie gebaseerd op parameters
def simulate_power(params):
    efficiency, temp_coeff = params
    return (
        2.8 * np.exp(-0.15 * (time_steps - 12) ** 2)
    ) * efficiency * (1 + temp_coeff * (temperature - 25))

# Objective functie: minimaliseer verschil tussen gemeten en gesimuleerde data
def objective_function(params):
    simulated = simulate_power(params)
    return -r2_score(measured_power, simulated)  # Negatieve R^2 om te maximaliseren

# Beginwaarden en marges
initial_guess = [baseline_efficiency, baseline_temp_coeff]
bounds = [(0.18, 0.22), (-0.005, -0.003)]  # Rendement: 18-22%, temp-coeff: -0.5% tot -0.3%

# Bereken R^2 vóór optimalisatie
initial_r2 = r2_score(measured_power, simulated_power)

# Optimalisatie
result = minimize(objective_function, initial_guess, bounds=bounds, method='L-BFGS-B')
optimal_params = result.x
final_r2 = -result.fun  # Negatief omgedraaid naar positief

# Gesimuleerd vermogen na optimalisatie
optimized_power = simulate_power(optimal_params)

# Bereken procentuele veranderingen
percentage_differences = [
    (optimal_params[i] - initial_guess[i]) / initial_guess[i] * 100 for i in range(len(parameters))
]

# Resultatentabel
results_table = pd.DataFrame({
    "Parameter": parameters,
    "Oorspronkelijke Waarde": [f"{initial_guess[0]*100:.2f}%", f"{initial_guess[1]*100:.2f}%"],
    "Aangepaste Waarde": [f"{optimal_params[0]*100:.2f}%", f"{optimal_params[1]*100:.2f}%"],
    "Verschil (%)": percentage_differences
})

# Toon resultaten in console
print("\nOptimalisatieresultaten Parameters:")
print(results_table)

# Toon R^2 vóór en na optimalisatie
print(f"\nR^2 vóór optimalisatie: {initial_r2:.4f}")
print(f"R^2 na optimalisatie: {final_r2:.4f}")

# Verbeterde grafieken voor rapportage
plt.figure(figsize=(14, 8))
plt.plot(time_steps, measured_power, label="Gemeten Vermogen", linestyle=":", color="blue", linewidth=3)
plt.plot(time_steps, simulated_power, label=f"Oorspronkelijke Simulatie (R^2={initial_r2:.2f})", linestyle="--", color="red", linewidth=3)
plt.plot(time_steps, optimized_power, label=f"Geoptimaliseerde Simulatie (R^2={final_r2:.2f})", linestyle="-", color="green", linewidth=3)
plt.xlabel("Tijd (uur)", fontsize=18)
plt.ylabel("Vermogen (kW)", fontsize=18)
plt.title("Vergelijking vóór en na Optimalisatie", fontsize=20)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.show()

# Residuen vóór en na optimalisatie
residuals_before = measured_power - simulated_power
residuals_after = measured_power - optimized_power

plt.figure(figsize=(14, 8))
plt.plot(time_steps, residuals_before, label="Residuen vóór Optimalisatie", linestyle="--", color="orange", linewidth=3)
plt.plot(time_steps, residuals_after, label="Residuen na Optimalisatie", linestyle="-", color="purple", linewidth=3)
plt.axhline(0, color="black", linestyle=":", linewidth=2)
plt.xlabel("Tijd (uur)", fontsize=18)
plt.ylabel("Residuen (kW)", fontsize=18)
plt.title("Residuen vóór en na Optimalisatie", fontsize=20)
plt.legend(fontsize=14)
plt.grid(True)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'numpy'